In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
# config
numEpochs = 4 # number of epochs to train the GPT+PT model
embeddingSize=512 # the hidden dimension of the representation of both GPT and PT
numPoints=20 # number of points that we are going to receive to make a prediction about f given x and y, if you don't know then use the maximum
numVars=2 # the dimenstion of input points x, if you don't know then use the maximum
numYs=1 # the dimension of output points y = f(x), if you don't know then use the maximum
blockSize = 100 # spatial extent of the model for its context
batchSize = 128
dataDir = 'D:/Datasets/Symbolic Dataset/Datasets'
dataInfo = 'XYE_{}Var_{}Points'.format(numVars, numPoints)
target = 'Skeleton' #'EQ'
dataFolder = '1-2Var_RandSupport_RandLength_-1to4_4.1to8_20Points'
addr = './SavedModels/bestModel/' # where to save model
maxNumFiles = 30

In [5]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars, target='EQ'):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1000,1000]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
        self.target = target
    
    def __len__(self):
        return len(self.data)-1

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        
        try:
            chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        except:
            print("Couldn't convert to json: {}".format(chunk))
            
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk[self.target]+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList 
        
        # make sure it is not more than what should be
        inputs = inputs[:self.block_size]
        outputs = outputs[:self.block_size]
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
            p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [6]:
import json
from tqdm import tqdm
import glob
def processDataFiles(files):
    text = ''""
    for f in tqdm(files):
        with open(f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            if lines[-1]==-1:
                lines = lines[:-1]
            text += lines #json.loads(line)       
            
    return text

In [7]:
path = '{}/{}/Train/*.json'.format(dataDir, dataFolder)
files = glob.glob(path)[:maxNumFiles]
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
text = text[:-1] if len(text[-1]) == 0 else text
train_dataset = CharDataset(text, blockSize, chars, target=target) 

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 40.59it/s]


data has 100000 examples, 55 unique.


In [8]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

inputs:tensor([22, 37, 45, 47,  3, 47, 41, 44,  3,  3, 24,  5, 50, 12,  6, 24,  4,  4,
         4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35])
id:15795
inputs:<cos(sin((C*x1+C)))_________________________________________________________________________________
outputs:cos(sin((C*x1+C)))>_________________________________________________________________________________
points:tensor([[ 0.8991,  1.2307,  0.4329,  1.9156, -0.4478,  1.5841,  1.5647,  0.9353,
         -0.7590, -0.3389,  2.6095,  0.8427,  3.5363, -0.7046,  1.9932,  1.0733,
         -0.3761, -0.0278, -0.3819,  3.4057],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0

In [9]:
path = '{}/{}/Val/*.json'.format(dataDir,dataFolder)
files = glob.glob(path)
textVal = processDataFiles([files[0]])
textVal = textVal.split('\n') # convert the raw text to a set of examples
val_dataset = CharDataset(textVal, blockSize, chars, target=target)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.99it/s]

data has 501 examples, 55 unique.


In [10]:
idx = np.random.randint(val_dataset.__len__())
inputs, outputs, points = val_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(-0.3858) tensor(456.9712)
id:348
inputs:<pow(((C*x1+C)/(C*x1+C)),5)_________________________________________________________________________
outputs:pow(((C*x1+C)/(C*x1+C)),5)>_________________________________________________________________________
points:tensor([[ 2.2489e+00, -2.5080e-01,  3.6675e+00,  1.8503e+00,  3.8860e-01,
          1.5595e+00, -3.8580e-01,  1.9170e+00,  6.7570e-01,  1.7317e+00,
         -4.5300e-02,  2.3056e+00,  7.0390e-01,  2.6804e+00,  1.6749e+00,
          5.7580e-01,  1.5970e-01,  1.9756e+00,  2.7611e+00,  2.7498e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.0007e+01,  5.4710e-01,  4.5697e+02,  2.0180e+01,  1.6408e+00,
          1.2249e+01,  4.3400e-01,  2.2628e+01,  2.6861e+00,  1.6462

In [11]:
path = '{}/{}/Test/*.json'.format(dataDir,dataFolder)
files = glob.glob(path)
textTest = processDataFiles(files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, blockSize, chars)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.79it/s]

data has 1001 examples, 55 unique.


In [12]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(-1000.) tensor(113.8771)
id:270
inputs:<-1.42*x1**2+4.09*x1*pow(1.73*x1+2.19,4)____________________________________________________________
outputs:-1.42*x1**2+4.09*x1*pow(1.73*x1+2.19,4)>____________________________________________________________
points:tensor([[ 3.4503e+00,  2.8384e+00,  3.1513e+00,  3.3307e+00,  3.5522e+00,
         -4.8980e-01,  3.5229e+00,  2.7493e+00,  3.8766e+00,  1.8728e+00,
          3.6572e+00,  9.2690e-01,  3.6781e+00,  3.1336e+00,  1.7712e+00,
          2.9500e-02, -3.6260e-01, -1.2290e-01,  2.2273e+00,  3.5315e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.0000e+03,  4.9921e+01, -7.1444e+02, -1.0000e+03, -1.0000e+03,
         -2.0236e+00, -1.0000e+03,  1.0945e+02, -1.0000e+03,  6.3736e

In [13]:
from mingpt.model import GPT, GPTConfig, PointNetConfig
pconf = PointNetConfig(embeddingSize=embeddingSize, 
                       numberofPoints=numPoints, 
                       numberofVars=numVars, 
                       numberofYs=numYs)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=embeddingSize, padding_idx=train_dataset.paddingID)
model = GPT(mconf, pconf)

05/24/2021 09:30:44 - INFO - mingpt.model -   number of parameters: 3.058740e+07


In [14]:
fName = '{}_SymbolicGPT_{}_{}_{}_MINIMIZE.txt'.format(dataInfo, 
                                             'GPT_PT_Summation', 
                                             'Padding',
                                             blockSize)

In [15]:
from mingpt.trainer import Trainer, TrainerConfig
import os

try: 
    os.mkdir(addr)
except:
    print('Folder already exists!')
    
ckptPath = '{}/{}.pt'.format(addr,fName.split('.txt')[0])
# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=numEpochs, batch_size=batchSize, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*blockSize,
                      num_workers=0, ckpt_path=ckptPath)
trainer = Trainer(model, train_dataset, val_dataset, tconf)

try:
    trainer.train()
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Folder already exists!


epoch 1 iter 124: train loss 0.42364. lr 5.906849e-04:  16%|███▉                     | 125/782 [00:51<04:28,  2.45it/s]

Input:tensor([22, 38, 50, 46,  3, 46, 45, 49,  3,  3, 24,  5, 50, 13,  6, 24,  4,  7,
        15,  4,  4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([ 3, 50, 46,  3,  3, 50,  5,  3,  3,  3,  5,  5, 12,  6, 24,  4,  4,  6,
         4,  4, 24,  3, 50,  6,  3, 12,  6,  3, 50, 50, 24, 24, 50,  6,  4, 24,
         6, 24,  4,  4,  4, 24,  6, 24,  6, 24,  4, 24,  3, 24, 12, 24, 24, 24,
        24, 24, 50,  6, 50,  4,  6, 24, 50,  4,  6, 24, 24,  6, 24,  4,  6,  4,
        24, 24,  4, 24, 24, 24,  4,  4,  6, 50, 24, 24,  4, 24,  4,  4, 24,  4,
        24,  4, 24,  4, 24, 50,  4, 24, 12, 50], device='cuda:0')
Input:<exp(pow((C*x2+C),4))_____________________________

epoch 1 iter 393: train loss 0.27773. lr 5.109429e-04:  50%|████████████▌            | 394/782 [02:41<02:38,  2.44it/s]

Input:tensor([22, 46, 45, 49,  3,  3,  3, 24,  5, 50, 12,  6, 24,  4, 10,  3, 24,  5,
        50, 12,  6, 24,  4,  4,  7, 15,  4, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([ 3, 45, 49,  3,  3,  3, 24,  5, 50, 12,  6, 24,  4, 10,  3, 24,  5, 50,
        12,  6, 24,  4,  4,  7, 16,  4, 23,  3, 46, 23,  4,  3,  4,  3,  4,  3,
        50,  3, 23, 23,  4,  3,  3,  3,  3,  3,  3, 24, 24, 50, 50, 24, 24, 24,
        24, 50, 24, 24, 24, 50, 24, 24, 24, 24, 24, 50, 50, 24, 24, 24, 50, 50,
        50, 24, 50, 50, 24, 12, 12, 12, 24, 12, 24, 24, 24, 24, 24, 50, 12, 24,
        24, 50, 24, 12, 12, 50, 50, 50, 12,  5], device='cuda:0')
Input:<pow(((C*x1+C)/(C*x1+C)),4)_______________________

epoch 1 iter 781: train loss 0.23591. lr 3.002414e-04: 100%|█████████████████████████| 782/782 [05:20<00:00,  2.44it/s]
05/24/2021 09:36:08 - INFO - mingpt.trainer -   test loss: 0.212383
05/24/2021 09:36:08 - INFO - mingpt.trainer -   saving ./SavedModels/bestModel//XYE_2Var_20Points_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE.pt
epoch 2 iter 781: train loss 0.23635. lr 6.000000e-05: 100%|█████████████████████████| 782/782 [05:21<00:00,  2.44it/s]
05/24/2021 09:41:30 - INFO - mingpt.trainer -   test loss: 0.186940
05/24/2021 09:41:30 - INFO - mingpt.trainer -   saving ./SavedModels/bestModel//XYE_2Var_20Points_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE.pt
epoch 3 iter 781: train loss 0.43338. lr 3.002414e-04: 100%|█████████████████████████| 782/782 [05:22<00:00,  2.42it/s]
05/24/2021 09:46:54 - INFO - mingpt.trainer -   test loss: 0.190196
epoch 4 iter 184: train loss 0.22081. lr 4.093130e-04:  24%|█████▉                   | 185/782 [01:16<04:08,  2.40it/s]

Input:tensor([22, 43, 45, 40,  3, 37, 45, 47,  3,  3, 24,  5, 50, 12,  6, 24,  4,  4,
         4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([47, 45, 40,  3, 47, 45, 47,  3,  3, 24,  5, 50, 13,  6, 24,  4,  4,  4,
        23, 23,  3,  3, 23, 23,  3, 50, 24, 24, 50, 46, 50,  3,  4, 23,  4,  3,
        50, 24, 23,  4,  4,  3, 50, 50, 24, 24, 24, 24,  6, 24, 24, 50, 24, 24,
        24, 24, 50, 24,  5, 24,  6, 24, 50, 24,  6, 24, 24, 24, 13, 24, 24,  5,
        50, 24, 24, 24,  6, 24,  4,  3, 50,  6, 24, 24,  3, 24, 24,  6,  4, 50,
         6,  6,  4,  6,  3,  3,  6, 50,  4, 24], device='cuda:0')
Input:<log(cos((C*x1+C)))_______________________________

epoch 4 iter 566: train loss 0.21207. lr 5.727678e-04:  72%|██████████████████       | 566/782 [03:53<01:30,  2.39it/s]

Input:tensor([22, 43, 45, 40,  3, 43, 45, 40,  3,  3, 24,  5, 50, 12,  6, 24,  4,  4,
         4, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([43, 45, 40,  3,  3, 45, 40,  3,  3, 24,  5, 50, 12,  6, 24,  4,  4,  4,
        23, 23, 23, 23, 45, 23, 23, 23, 23, 23,  4, 23, 23, 23,  4, 23, 23, 23,
        23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
        23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 23, 23, 23, 23, 14, 23, 23, 23,
        23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,  3,  3, 23, 23,
        23, 23, 23, 23, 23, 23, 23, 23, 24, 24], device='cuda:0')
Input:<log(log((C*x1+C)))_______________________________

epoch 4 iter 781: train loss 0.22241. lr 5.999996e-04: 100%|█████████████████████████| 782/782 [05:21<00:00,  2.43it/s]
05/24/2021 09:52:16 - INFO - mingpt.trainer -   test loss: 0.258088


In [16]:
# load the best model
model.load_state_dict(torch.load(ckptPath))
model = model.eval().to(trainer.device)

In [17]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [18]:
# class pointsDataset(Dataset):

#     def __init__(self, data):
#         # data should be a list of x,y pairs
#         self.x = data[0] # it should be a list
#         self.y = data[1] # it should be a list
#         self.threshold = [-1000,1000]
    
#     def __len__(self):
#         return len(self.y)

#     def __getitem__(self, idx):
#         # grab an example from the data
#         x = self.x[idx] 
#         y = self.y[idx] 
        
#         p = x+[y]
        
#         p = torch.tensor(p, dtype=torch.float)
#         p = torch.nan_to_num(p, nan=0.0, 
#                              posinf=self.threshold[1], 
#                              neginf=self.threshold[0])
#         p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
#         p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
        
#         return p
    
# # train a mlp to find the constants
# data = pointsDataset((t['X'],t['Y']))
# loader = torch.utils.data.DataLoader(
#                                 data, 
#                                 shuffle=False, 
#                                 pin_memory=True,
#                                 batch_size=batchSize,
#                                 num_workers=0)
# class MLP(nn.Module):
#     def __init__(self, inputSize, outputSize):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(inputSize, 100),
#             nn.ReLU(),
#             nn.Linear(100, outputSize)
#         )

#     def forward(self, x):
#         c = self.layers(x)
#         return c
    
#     def loss(self, constants, eq, X, Y):
#         # constants is the output of the model
#         err = torch.zeros(len(constants)) # batch_size
        
#         # sample number of points
#         indexes = []
#         numberSamples = 10
#         while len(indexes) != numberSamples:
#             randNum = np.random.randint(len(batch))
#             indexes.append(randNum)
#         X = X[indexes,:]
#         Y = Y[indexes,:]
        
#         # replace the constants with their predicted values
#         for idx, const in enumerate(constants):
#             eq = eq.replace('C','{}').format(*const.tolist())
        
#         # calculate the error for a limited number of points, approximate the error
#         for x,y in zip(X,Y):
#             # replace variables with their values
#             for i,e in enumerate(x):
#                 eqTemp = eq.replace('x{}'.format(i+1), str(e.item()))
                
#             # calculate the error
#             yHat = eval(eqTemp)
#             err[idx] += (y.item()-yHat)**2
#         err[idx] /= numberSamples
            
#         print(err.shape, constants.shape)
#         return err
    
# c = [0 for i,x in enumerate(predicted) if x=='C']
# network = MLP(numVars+numYs, len(c))
# cHat = network(batch)
# err = network.loss(cHat, predicted, batch[:,:numVars], batch[:,-numYs:])

In [19]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
from gp_model import Genetic_Model
from mlp_model import MLP_Model
from scipy.optimize import least_squares, newton, minimize

    
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)

testRange = [3.1,6.0]
numTestPoints = 10
#test = np.linspace(3.1,6.0,numTestPoints)

gpm = Genetic_Model(n_jobs=-1)
mlp = MLP_Model()

resultDict = {}
try:
    with open(fName, 'w', encoding="utf-8") as o:
        modelName = 'SymbolicGPT'
        resultDict[fName] = {modelName:[],
                             'GP':[],
                             'MLP':[]}

        for i, batch in enumerate(loader):
                
            inputs,outputs,points = batch

            print('Test Case {}.'.format(i))
            o.write('Test Case {}/{}.\n'.format(i,len(textTest)-1))

            t = json.loads(textTest[i])

            inputs = inputs[:,0:1].to(trainer.device)
            points = points.to(trainer.device)
            outputsHat = sample(model, inputs, blockSize, points=points,
                          temperature=1.0, sample=True, 
                          top_k=40)[0]

            # filter out predicted
            target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
            predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])

            target = target.strip(train_dataset.paddingToken).split('>')
            target = target[0] if len(target[0])>=1 else target[1]
            target = target.strip('<').strip(">")
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            
            print('Target:{}\nSkeleton:{}'.format(target, predicted))
            
            o.write('{}\n'.format(target))
            o.write('{}:\n'.format(modelName))
            o.write('{}\n'.format(predicted))

            # train a regressor to find the constants (too slow)
            c = [1 for i,x in enumerate(predicted) if x=='C']            
            def lossFunc(constants, eq, X, Y):
                err = 0
                eq = eq.replace('C','{}').format(*constants)

                for x,y in zip(X,Y):
                    eqTemp = eq + ''
                    for i,e in enumerate(x):
                        eqTemp = eqTemp.replace('x{}'.format(i+1), str(e))
                    try:
                        yHat = eval(eqTemp)
                    except:
                        yHat = 100
                    err += (y-yHat)**2
                err /= len(Y)
                return err
            
            try:
                if len(c) == 0:
                    pass # do nothing
                else:
#                     cHat = least_squares(lossFunc, c, ftol=1e-3,
#                                          args=(predicted, t['X'], t['Y']))
                    
                    # for easier comparison, we are using minimize package  
                    cHat = minimize(lossFunc, c,
                                   args=(predicted, t['X'], t['Y'])) 
            
#                     cHat= newton(lossFunc, c, maxiter=100,
#                                  args=(predicted, t['X'], t['Y']))
                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except:
                print('Wrong Equation:{}'.format(predicted))
                raise
                predicted = 0

            # TODO: let's enjoy GPU

            print('Skeleton+LS:{}'.format(predicted))

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
                try:
                    eqTmp = target + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    YEval = eval(eqTmp)
                    YEval = 0 if np.isnan(YEval) else YEval
                    YEval = 100 if np.isinf(YEval) else YEval
                except:
                    YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
                Ys.append(YEval)
                try:
                    eqTmp = predicted + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    Yhat = eval(eqTmp)
                    Yhat = 0 if np.isnan(Yhat) else Yhat
                    Yhat = 100 if np.isinf(Yhat) else Yhat
                except:
                    Yhat = 100
                Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats)

            if type(err) is np.complex128 or np.complex:
                err = abs(err.real)

            resultDict[fName][modelName].append(err)

            o.write('{}\n{}\n\n'.format( 
                                    predicted,
                                    err
                                    ))

            print('Err:{}'.format(err))
            
            # Calculate error for baselines

    #         # tokenize to get input x, input y, and true eqn
    #         train_data_x = t["X"]
    #         train_data_y = t["Y"]
    #         test_data_x = t["XT"]
    #         test_data_y = t["YT"]

    #         # train MLP model
    #         mlp.reset()
    #         model_eqn, _, best_err = mlp.repeat_train(
    #                                                 train_data_x, 
    #                                                 train_data_y,
    #                                                 test_x=test_data_x, 
    #                                                 test_y=test_data_y,                                     
    #                                                 verbose=False)
    #         test_y_hat = mlp.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("{}: {}".format(mlp.name, model_eqn)[:550])
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['MLP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('MLP', 
    #                                    err,
    #                                    model_eqn))

    #         # train GP model
    #         gpm.reset()
    #         model_eqn, _, best_err = gpm.repeat_train(train_data_x, train_data_y,
    #                                                 test_x=test_data_x, test_y=test_data_y,
    #                                                 verbose=False)
    #         print("{}: {}".format(gpm.name, model_eqn)[:550])
    #         test_y_hat = gpm.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['GP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('GP', 
    #                                    err,
    #                                    model_eqn))
            print('') # just an empty line
    print('Avg Err:{}'.format(np.mean(resultDict[fName][modelName])))
    
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Test Case 0.
Target:exp(pow(x1-0.98,4))
Skeleton:pow((C*x1+C),3)
Skeleton+LS:pow((1.0*x1+1.0),3)
Err:10.068103096650116

Test Case 1.


C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


Target:1-0.23/(x1*sin(2.72*x1))
Skeleton:(((C*x1+C)+(C*x1+C))/sin((C*x1+C)))
Skeleton+LS:(((1.1298427807825395*x1+-2.453823046958435)+(1.129762242988748*x1+-2.453823046948585))/sin((1.3644376493132768*x1+-0.005010648407750828)))
Err:3138.0381236127373

Test Case 2.
Target:1.3200
Skeleton:cos(sin((C*x1+C)))
Skeleton+LS:cos(sin((-4.141124875799737e-08*x1+3.1415927480961203)))
Err:0.058769513314972946

Test Case 3.
Target:-2.3935*x1-1.59
Skeleton:(((C*x1+C)*(C*x1+C))+(C*x1+C))
Skeleton+LS:(((-4.4913349813040915*x1+12.343406864603583)*(0.3954864535413569*x1+3.106602629718869))+(4.375813879834977*x1+-40.96332014346943))
Err:7130.593827374146

Test Case 4.
Target:0.2039*exp(1.55*x1)
Skeleton:(pow((C*x1+C),4)+((C*x1+C)/(C*x1+C)))
Skeleton+LS:(pow((0.018503084953612*x1+-4.552012326465468),4)+((34.02238401815636*x1+-153.46062635525772)/(-0.074828233320256*x1+0.3574577625170874)))
Err:1.72757533462909

Test Case 5.
Target:-0.7182*x1*exp(x1)
Skeleton:sin(exp((C*x1+C)))
Skeleton+LS:sin(exp((1.0112

C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


Target:sin(x1**2-2.1*x1+0.54)
Skeleton:cos(pow((C*x1+C),5))
Skeleton+LS:cos(pow((1.9690076589056282*x1+1.284804650063654),5))
Err:1.2310363344955528

Test Case 16.
Target:-2.15*x1/(-1.68*x1-0.09)-log(x1)
Skeleton:exp(((C*x1+C)/(C*x1+C)))
Skeleton+LS:exp(((-2.2278617807584093*x1+2.6599076802786974)/(-1.1975484549416757*x1+4.395192322145042)))
Err:371345298.57009953

Test Case 17.


...

C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


Skeleton+LS:(sin((291.25029665950444*x2+940.2882143591016))*log((83.67045173714628*x2+-309.3391526455153)))
Err:16.303096379792585

Test Case 826.
Target:0.0679*exp(-0.87*x1)
Skeleton:pow((C*x1+C),5)
Skeleton+LS:pow((-0.10206467221030921*x1+0.5878990446437753),5)
Err:3.5129517261790756e-07

Test Case 827.
Target:cos(1.03*x2/x1)
Skeleton:log(((C*x2+C)-(C*x1+C)))
Skeleton+LS:log(((1.0*x2+1.0)-(1.0*x1+1.0)))
Err:0.7663108768669533

Test Case 828.
Target:cos(2*x2)
Skeleton:exp((C*x2+C))
Skeleton+LS:exp((0.8181398647855889*x2+-3.7910248133678133))
Err:51.14803910775753

Test Case 829.
Target:-0.9*x1*cos(2.2*x2)-2.76*x2+2.63
Skeleton:(((C*x2+C)+(C*x1+C))-log((C*x2+C)))


C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


Skeleton+LS:(((482.3780714453754*x2+-70.17772258708703)+(-1293.4400819114492*x1+-70.17772258708744))-log((-1053.5588346241368*x2+-1076.127592664076)))
Err:214.20558668076964

Test Case 830.
Target:log(log(x1))
Skeleton:log(cos((C*x2+C)))
Skeleton+LS:log(cos((1.0*x2+1.0)))
Err:1.4783303619610568

Test Case 831.
Target:cos(0.14*exp(x1))
Skeleton:log(sin((C*x1+C)))
Skeleton+LS:log(sin((1.0*x1+1.0)))
Err:0.6897118166041177

Test Case 832.
Target:sin(x1)-sin(1.06*x1-2.31)
Skeleton:sin((C*x2+C))
Skeleton+LS:sin((0.10228937011641044*x2+0.19727903877336037))
Err:1.028958133569202

Test Case 833.
Target:sin(cos(x1))
Skeleton:sin((C*x2+C))
Skeleton+LS:sin((1.4771790268746652*x2+0.7500956975708617))
Err:0.8985935093042731

Test Case 834.
Target:pow(exp(2.8*x2),6)
Skeleton:(pow((C*x1+C),6)-pow((C*x1+C),6))
Wrong Equation:(pow((C*x1+C),6)-pow((C*x1+C),6))


OverflowError: (34, 'Result too large')

In [ ]:
# plot the error frequency for model comparison
from matplotlib import pyplot as plt
num_eqns = len(resultDict[fName]['SymbolicGPT'])
num_vars = pconf.numberofVars

models = list(key for key in resultDict[fName].keys() if len(resultDict[fName][key])==num_eqns)
lists_of_error_scores = [resultDict[fName][key] for key in models if len(resultDict[fName][key])==num_eqns]
linestyles = ["-","dashdot","dotted","--"]

eps = 0.00001
y, x, _ = plt.hist([np.log([max(min(x+eps, 1e5),1e-5) for x in e]) for e in lists_of_error_scores],
                   label=models,
                   cumulative=True, 
                   histtype="step", 
                   bins=2000, 
                   density=True,
                   log=False)
y = np.expand_dims(y,0)
plt.figure(figsize=(15, 10))

for idx, m in enumerate(models): 
    plt.plot(x[:-1], 
           y[idx] * 100, 
           linestyle=linestyles[idx], 
           label=m)

plt.legend(loc="upper left")
plt.title("{} equations of {} variables".format(num_eqns, num_vars))
plt.xlabel("Log of Relative Mean Square Error")
plt.ylabel("Normalized Cumulative Frequency")

name = '{}.png'.format(fName.split('.txt')[0])
plt.savefig(name)

In [ ]:
# benchmarks